In [1]:
#dependencies

import pandas as pd
import numpy as np
import requests
import json

moviedbapi = "87594855563381b8f9fa3b3224c9f19f"
idURL = "https://api.themoviedb.org/3/search/movie?query="
idURL1 = "&api_key="
detailURL = "https://api.themoviedb.org/3/movie/"
detailURL1 = "?api_key="

In [2]:
omdbapi = "e8913e4c"
omdb1 = "http://www.omdbapi.com/?t="
omdb2 = "&apikey="

In [3]:
#import csv file as pd dataframe

csvPath = 'movies.csv'
opening = pd.read_csv(csvPath)
print(len(opening))
opening.head()

1500


,release,title,genre,budget,opening,domestic,worldwide
0,20170120,Split,Horror,5000000,40010975,138141585,278449984
1,20170120,xXx: Return of Xander Cage,Action,85000000,20130142,44898413,345114933
2,20170127,A Dog’s Purpose,Drama,25000000,18222810,64321890,205272612
3,20170106,Underworld: Blood Wars,Action,0,13688751,30405609,81096784
4,20170127,Resident Evil: The Final Chapter,Action,40000000,13601682,26844692,307353906


In [4]:
#filters out movies that made at least $10000 in the first weekend

opening = opening.loc[(opening['opening']>999)]
print(len(opening))
opening.head()

642


,release,title,genre,budget,opening,domestic,worldwide
0,20170120,Split,Horror,5000000,40010975,138141585,278449984
1,20170120,xXx: Return of Xander Cage,Action,85000000,20130142,44898413,345114933
2,20170127,A Dog’s Purpose,Drama,25000000,18222810,64321890,205272612
3,20170106,Underworld: Blood Wars,Action,0,13688751,30405609,81096784
4,20170127,Resident Evil: The Final Chapter,Action,40000000,13601682,26844692,307353906


In [5]:
#filters our rows with the budgets already populated and saves for later

budgetDF = opening.loc[(opening["budget"] > 0)]
print(len(budgetDF))
budgetDF.head()

132


,release,title,genre,budget,opening,domestic,worldwide
0,20170120,Split,Horror,5000000,40010975,138141585,278449984
1,20170120,xXx: Return of Xander Cage,Action,85000000,20130142,44898413,345114933
2,20170127,A Dog’s Purpose,Drama,25000000,18222810,64321890,205272612
4,20170127,Resident Evil: The Final Chapter,Action,40000000,13601682,26844692,307353906
5,20170113,The Bye Bye Man,Thriller/Suspense,7400000,13501349,22395806,25863405


In [ ]:
nobudgetDF = opening.loc[(opening["budget"] == 0)]
print(len(nobudgetDF))
nobudgetDF.head()

510


,release,title,genre,budget,opening,domestic,worldwide
3,20170106,Underworld: Blood Wars,Action,0,13688751,30405609,81096784
7,20170113,Sleepless,Thriller/Suspense,0,8344128,20783704,27224309
8,20170127,Gold,Drama,0,3471316,7227038,11478501
10,20170125,Raees,Action,0,1798379,2369469,5429008
11,20170120,The Resurrection of Gavin Stone,Comedy,0,1206771,2303792,2303792


In [ ]:
#get ids
idList = []
df = []

for index, row in nobudgetDF.iterrows():
#     print('--------------------------------------------------------------------------------')
    movie = row['title']
#     finalMovie = movie.replace(":","")
#     finalMovie = movie.replace(" ","%20")
    openGross = row['opening']
    domestic = row['domestic']
    worldwide = row['worldwide']
    date = row['release']
    genre = row['genre']
    finalURL = idURL + movie + idURL1 + moviedbapi
#     print (finalURL)
    json_file = requests.get(finalURL).json()
    moviesJSON = json_file['results']
#     print (moviesJSON)
#     print (len(json_file['results']))
    
    try:
        dates = []
        for n in np.arange(len(moviesJSON)):
#             print (n)
            dt = moviesJSON[n]['release_date']
            dt = int(dt.replace("-",""))
#             print (dt)
            dates.append(dt)
#             print (f"success {n}")
#         print (dates)      
#         print (recent)
    except:
        pass
#         print ("no dates")
    try:
        for number in dates:
#             print (number)
            if date-30 <= number and number <= date + 30:
#                 print("DateMatch")
                ind = dates.index(number)
#                 print (ind)
#                 print (moviesJSON)
                recentMovie = moviesJSON[ind]
                idNumber = recentMovie['id']
#                 print (f"success {n}.{n}")
#                 movie = recentMovie['original_title']
#                 print(f'{movie} / {date} / {ind} / {idNumber} / {openGross}')
                movieDict = {"release":date,"title":movie, "gross":openGross, "id_number":idNumber, "domestic":domestic, 
                             "worldwide":worldwide, "genre":genre}
                df.append(movieDict)
            else:
                continue
    
    except:
        print ("no id")
    
idDF = pd.DataFrame(df)
print (len(idDF))
pd.options.display.max_rows = 999
idDF.head()

In [ ]:
#get budgets
budgets = []
for index, row in idDF.iterrows():
    genre = row['genre']
    gross = row['gross']
    title = row['title']
    domestic = row['domestic']
    worldwide = row['worldwide']

    try:
        
        detailsURL = detailURL + str(row['id_number']) + detailURL1 + moviedbapi
#     print (detailsURL)
        details = requests.get(detailsURL).json()
        budget = details['budget']
        release = details['release_date']
#         print (f"Budget Found = {budget}")
        fullDict = {"title": title, "opening":gross, "budget":budget, "domestic":domestic, 
                    "genre":genre, "release":release, "worldwide":worldwide}
        budgets.append(fullDict)
    except:
        pass
#     print (genre)
    
addbudgetDF = pd.DataFrame(budgets)
addbudgetDF = addbudgetDF.loc[(addbudgetDF['budget']>0)]
print(len(addbudgetDF))
addbudgetDF.head()

In [ ]:
# add budget to original dataframe
# release	title	genre	budget	opening	domestic	worldwide
budgetFinal = []

for index, row in budgetDF.iterrows():
    release = row['release']
    title = row['title']
    genre = row['genre']
    budget = row['budget']
    opening = row['opening']
    domestic = row['domestic']
    worldwide = row['worldwide']
    moviesData = {"release": release, "title":title, "genre":genre, "budget":budget, 
                  "opening":opening, "domestic":domestic, "worldwide":worldwide}
    budgetFinal.append(moviesData)
    
for index, row in addbudgetDF.iterrows():
    release = row['release']
    title = row['title']
    genre = row['genre']
    budget = row['budget']
    opening = row['opening']
    domestic = row['domestic']
    worldwide = row['worldwide']
    moviesData = {"release": release, "title":title, "genre":genre, "budget":budget, 
                  "opening":opening, "domestic":domestic, "worldwide":worldwide}
    budgetFinal.append(moviesData)
    
moviesDF = pd.DataFrame(budgetFinal)
print (len(moviesDF))
moviesDF

In [ ]:
# get ratings
moviesFinal =[]
for index, row in moviesDF.iterrows():
    release = row['release']
    title = row['title']
    genre = row['genre']
    budget = row['budget']
    opening = row['opening']
    domestic = row['domestic']
    worldwide = row['worldwide']
    omdbURL = omdb1 + title + omdb2 + omdbapi
#     print (omdbURL)
    ratingJSON = requests.get(omdbURL).json()
    ratings = []
    finalRatings = []
    try:
        
        ratingJSON = ratingJSON['Ratings']
#         print (ratingJSON)
        for x in np.arange(len(ratingJSON)):
            value = ratingJSON[x]['Value']
            ratings.append(value)
#         print (ratings)
        for number in ratings:
#             print (number)
#             print (number.replace("'",""))
            numbers = number.replace("'","")
#             print (numbers)
            numbers = numbers.replace("/100","")
            numbers = numbers.replace("/10","")
            numbers = numbers.replace(".","")
            numbers = numbers.replace("%","")
            finalRatings.append(int(numbers))
    except:
        finalRatings = 0
    try:
        avgRating = np.mean(finalRatings).round(2)
#         print (avgRating)
    except:
        avgRating = 0
    
    moviesFinalDict = {"release": release, "title":title, "genre":genre, "budget":budget, 
                      "opening":opening, "domestic":domestic, "worldwide":worldwide, "value": avgRating}
    moviesFinal.append(moviesFinalDict)
moviesFinalDF = pd.DataFrame(moviesFinal)

moviesFinalDF = moviesFinalDF[['title','genre','release','value','budget','opening','domestic','worldwide']]

moviesFinalDF.to_csv('moviesFinal.csv', encoding='utf-8', index=False)
moviesFinalDF